In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [2]:
"""
conversion command:

nccopy -k 3 relocatable_Grenville.nc relocatable_Grenville_nc4.nc 
"""

'\nconversion command:\n\nnccopy -k 3 relocatable_Grenville.nc relocatable_Grenville_nc4.nc \n'

In [3]:
file_path = "relocatable_Grenville_nc4.nc"

In [4]:
ds = xr.open_dataset(file_path)
ds

<xarray.Dataset>
Dimensions:  (time: 3, ncell: 20053)
Coordinates:
  * time     (time) datetime64[ns] 2023-06-27T11:00:00 ... 2023-06-27T13:00:00
    lon      (ncell) float32 ...
    lat      (ncell) float32 ...
Dimensions without coordinates: ncell
Data variables:
    tm       (time, ncell) float32 ...
    u        (time, ncell) float32 ...
    v        (time, ncell) float32 ...
Attributes:
    title:      Surface winds and temperature
    version:    NETCDF3_CLASSIC
    source:     DFO
    reference:  Tue Jul  4 17:59:17 UTC 2023

In [5]:
longitude = np.unique(ds.lon.data)
latitude = np.unique(ds.lat.data)
time = np.array([(t - np.datetime64('1950-01-01T00:00:00Z')) / np.timedelta64(1, 's') for t in ds.time.data])

output_coords={
    "longitude": longitude,
    "latitude": latitude,
    "time": time,
}

output_attrs = ds.attrs

/tmp/ipykernel_11251/1018241735.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  time = np.array([(t - np.datetime64('1950-01-01T00:00:00Z')) / np.timedelta64(1, 's') for t in ds.time.data])


In [6]:
ds_variables = list(ds.keys())

output_variables = {
    variable: (
        ["time", "latitude", "longitude"],
        np.full((ds.time.size, latitude.size, longitude.size), np.nan),
        ds[variable].attrs,
    )
    for variable in ds_variables
}

In [7]:
for n in ds.ncell.data:
    lat = ds.lat.data[n]
    lon = ds.lon.data[n]

    lat_idx = np.argwhere(latitude == lat).flatten()
    lon_idx = np.argwhere(longitude == lon).flatten()

    for variable in output_variables.items():
        input_data = ds[variable[0]].data[:,n]
        output_shape = variable[1][1][:, lat_idx, lon_idx].shape
        variable[1][1][:, lat_idx, lon_idx] = input_data.reshape(output_shape)

In [8]:
nc_out = xr.Dataset(
    data_vars=output_variables, coords=output_coords
)